In [1]:
from bbi.models import BBI

In [2]:
env = BBI()
env.reset()

({'position': array([0.], dtype=float32),
  'status_indicator': array([5.], dtype=float32),
  'prize_indicators': array([0., 0.], dtype=float32)},
 {})

In [5]:
obs, _, _, _, _ = env.step(1)
obs

{'position': array([1.], dtype=float32),
 'status_indicator': array([5.], dtype=float32),
 'prize_indicators': array([0., 0.], dtype=float32)}

In [6]:
env.get_next_bounds(obs)

(array([[ 0.,  0.,  0.,  0.],
        [ 2., 10.,  0.,  0.]]),
 [-1.0, -1.0])

In [7]:
env.state_bounding_box

array([[ 0.,  0.,  0.,  0.],
       [ 2., 10.,  0.,  0.]])

In [1]:
from bbi.environments import GoRight

In [2]:
env = GoRight()
env.reset()

({'position': array([0.1597956], dtype=float32),
  'status_indicator': array([4.2896934], dtype=float32),
  'prize_indicators': array([-0.09481507, -0.13065626], dtype=float32)},
 {})

In [3]:
env.step(1)

({'position': array([1.1597956], dtype=float32),
  'status_indicator': array([9.289694], dtype=float32),
  'prize_indicators': array([-0.09481507, -0.13065626], dtype=float32)},
 -1.0,
 False,
 False,
 {})

In [7]:
env.tracker.history[-1].state

State(position=0, previous_status_indicator=np.int64(5), current_status_indicator=np.int64(5), prize_indicators=array([0., 0.]), offset=array([ 0.15979559, -0.71030643, -0.71030643, -0.09481507, -0.13065626]), mask=array([ True, False,  True,  True,  True]))

In [5]:
env.tracker.history[-1].next_state

State(position=np.int64(1), previous_status_indicator=np.int64(5), current_status_indicator=10, prize_indicators=array([0, 0]), offset=array([ 0.15979559, -0.71030643, -0.71030643, -0.09481507, -0.13065626]), mask=array([ True, False,  True,  True,  True]))